### Introdução
Essa é uma interface gráfica para acessar a base de dados dos experimentos no qcodes. Basta selecionar o experimento e o dataset dele(no Qcodes um experimento = grupo de datasets, datasets = dados de uma run) que quer plotar e/ou exportar como um arquivo JSON e visualizar o gráfico. Que aparecerão nas próprias janelas. Há também uma opção de gráfico em tempo real que precisa ser trabalhada para aceitar mais tipos de gráfico mas preechidas a condição de usar só duas variáveis está funcionando. 

### Instruções

Você precisa ter o Qcodes e o pysimpleGui para ele funcionar. Com isso de um Menu/Cell/run all (a intenção e deixar isso como um arquivo de python para servir de script pela linha de comando). As janelas vão aparecer começando com a database com experimentos carregados. Não dá pra testar esse caderno bem sem uma database do qcodes preechida com algo.

### To do
- A plotagem com  o plot_dataset não funciona com uma janela aberta ao mesmo tempo.
- O progress plot é para jupyter notebook que não é como esse programa vai ser entregue
- Tentar colocar tudo em uma janela com auxilio da celula abaixo:

In [ ]:
#exp é um elemento do tipo Combo atualiza os valores da droplist
window['exp'].Update(value=' ', values=[5,6,7])

#enable_event para poder usar um evento das listas com a key='-COLOR-' por exemplo
[sg.Listbox(choices, size=(15, len(choices)), key='-COLOR-', enable_events=True)]

In [5]:
%matplotlib qt
#%matplotlib inline

#GUI
import PySimpleGUI as sg
import sys
import json

import numpy as np
import qcodes as qc
from qcodes.logger.logger import start_all_logging
from qcodes.dataset.plotting import plot_dataset
from jupyterplot import ProgressPlot
import qcodes as qc
import numpy as np
import time
import matplotlib.pyplot as plt

#sg.theme('DarkAmber')
sg.theme('Dark Blue 3')

start_all_logging()

Activating auto-logging. Current session state plus future input saved.
Filename       : /home/daniel/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /home/daniel/.qcodes/logs/200628-24249-qcodes.log


In [40]:
defaultDtbs="./teste.db"

dtbs=defaultDtbs
#Criar um database
qc.initialise_or_create_database_at(dtbs)
#mostrar os experimentos disponiveis
experiments = qc.experiments() 
#cria lista com o nome dos experimentos
expList=[]
for exp in experiments:
    expList.append(exp.name)
  

# conteudo da janela 
layout = [  
            [sg.Text('Selecione a Database'), sg.InputText(key="dtbs", default_text=defaultDtbs, enable_events=True), sg.FileBrowse(),sg.Button('Update Database')],
            [sg.Text('Experimento'), sg.Combo(expList, key="exp", enable_events=True)],
            [sg.Text('Dataset'), sg.Combo([], key="dts",size=(50,10), enable_events=True, disabled=True)],
            [sg.Text('Exportar'), sg.InputText(key="save"), sg.FileSaveAs(),sg.Button('Save', disabled=True)],
            [sg.Button('Quick Plot', disabled=True)],
            [sg.Button('Continue') , sg.Button('Cancel')]]
  
# Cria a janela
window = sg.Window('Leitor Plotter', layout)    
# Loop de eventos para pegar os valores e fazer acoes
while True:
    event, values = window.read()
    if event in (None, 'Cancel'):# Termina a execucao do programa
        window.close()
        sys.exit()
        
    elif (event == "Update Database"):
        #eventos caso escolha nova database
        dtbs=values["dtbs"]
        #Criar um database
        qc.initialise_or_create_database_at(dtbs)
        #mostrar os experimentos disponiveis
        experiments = qc.experiments() 
        #cria lista com o nome dos experimentos
        expList=[]
        for exp in experiments:
            expList.append(exp.name)
        window['exp'].Update(value=' ', values=expList)
        #desabilita opcoes que dependem do experimento
        window['dts'].Update(disabled=True)
        window['Save'].Update(disabled=True)
        window['Quick Plot'].Update(disabled=True)
        
    elif (event=='exp'):
        expName=values["exp"]
        #eventos ao escolher um novo experimento
        #Definir um experimento escolhido pelo usuario pelo nome
        #e mostrar datasets do experimento escolhido
        exp = qc.load_experiment_by_name(expName)

        #pega tabela em texto com os datasets do experimento e transforma em string
        table=str(exp)
        #tranforma table em um vetor com as linhas
        lines=table.splitlines()
        #remove a primeira e segunda linhas
        del lines[0]
        del lines[0]
        #o vetor lines sera usado como parametro para escolher o dataset na janela
        window['dts'].Update(value=' ', values=lines) 
        #habilita escolha da database e desabilita do resto
        window['dts'].Update(disabled=False)
        window['Save'].Update(disabled=True)
        window['Quick Plot'].Update(disabled=True)
        
        
    elif(event=="dts"):
        #ao selecionar a dataset
        #pegar o valor do vetor que vem da janela
        stringdts=values["dts"]
        #para pegar o captured runID a partir do string de selecao
        #particiona o vetor no primeiro '-' que aparece e pega a primeira parte[0]
        dts = stringdts.partition('-')[0]
        #converte de string para int
        dts=int(dts)
        #escolher dataset e carregar para plotar
        dataset = qc.load_by_run_spec(experiment_name=exp.name,  captured_run_id=dts)
       #habilita salvar o dataset e o quickplot
        window['Save'].Update(disabled=False)
        window['Quick Plot'].Update(disabled=False)
         
            
    elif (event=='Quick Plot'):
         #faz um plot da dataset inteira de preview
        #plt.show(block=False)
        plot_dataset(dataset)
        window.close()
        # plt.draw()
        #plt.pause(0.001)
        
    elif (event == 'Save'):	# salva as entradas da janela para um arquivo em json
        f = open(values["save"], "w+")
        #remove as celulas com o caminho do arquivo da variavel ou seja sem mais uso
        f.write(json.dumps(dataset))
        f.close
        
    elif (event == 'Continue'): #Continua a execucao do programa
          break  
    
window.close()


[osc_teste dut#osc teste 1#1@/home/daniel/Documents/Estudos/1Graduacao/Iniciação Cientifica - Computação Quântica/measurement-instruments-python/teste.db
--------------------------------------------------------------------------------------------------------------------------------------------------------
1-results-1-SignalGenerator1_freq,gain-500
2-results-2-SignalGenerator1_freq,phaseGain-500
3-results-3-SignalGenerator1_freq,phaseGain-500
4-results-4-SignalGenerator1_freq,gain-500
5-results-5-SignalGenerator1_freq,phaseGain-500
6-results-6-SignalGenerator1_freq,gain-500
7-results-7-SignalGenerator1_freq,phaseGain-500
8-results-8-SignalGenerator1_freq,gain-500
9-results-9-SignalGenerator1_freq,phaseGain-500, gravitacao quantica#osc teste 1#2@/home/daniel/Documents/Estudos/1Graduacao/Iniciação Cientifica - Computação Quântica/measurement-instruments-python/teste.db
-------------------------------------------------------------------------------------------------------------------------

SystemExit: 

2020-06-28 20:33:13,578 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 99 ¦ /home/daniel/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [8]:
#pega a dependencias de dados (quais sao as variaveis independentes x, t e sua dependente y)
interdeps = dataset.description.interdeps
interdeps.dependencies

#dados dependentes y
#lista de parametros no dataset
PrmDep = list(interdeps.dependencies.keys())

#loop que lista nomes dos parametros no dataset
PDnames=[]
for ParY in PrmDep:
    PDnames.append(ParY.name)
print(PDnames)

['gain']


In [14]:
#janela para escolher variavel plotada exemplo(ganho ou fase)
layout = [  
            [sg.Text('Para ver o progress plot continue daqui')],
            [sg.Text('Escolher variável de saída'), sg.Combo(PDnames, key="PDnames")],
            [sg.Button('Continue') , sg.Button('Cancel')]]
# Cria a janela
window = sg.Window('Leitor Plotter', layout)


# Loop de eventos para pegar os valores e fazer acoes
while True:
    event, values = window.read()
    if event in (None, 'Cancel'):# Termina a execucao do programa
        window.close()
        sys.exit()
        
    elif (event == 'Continue'): #Continua a execucao do programa
        break
        
window.close()


#fecha plot
plt.close()

PrmYname=values["PDnames"]

SystemExit: 

2020-06-12 21:28:33,448 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 99 ¦ /home/daniel/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [10]:
#pega todos os parametros para essa variavel y e lista os nomes com keys()
PInames=list(dataset.get_parameter_data()[PrmYname].keys())
#remove a variavel dependente da lista (y)
PInames.remove(PrmYname); PInames

['SignalGenerator1_freq']

In [11]:
#janela para escolher variavel plotada

layout = [  
            [sg.Text('Escolher variavel de entrada'), sg.Combo(PInames, key="PInames")],
            [sg.Text('Intervalo de x'),  sg.InputText(key="inter1x", default_text="0"),  sg.InputText(key="inter2x", default_text="500")],
            [sg.Text('Intervalo de y'),  sg.InputText(key="inter1y", default_text="-200"),  sg.InputText(key="inter2y", default_text="200")],
            [sg.Button('Continue') , sg.Button('Cancel')]]
# Cria a janela
window = sg.Window('Leitor Plotter', layout)


# Loop de eventos para pegar os valores e fazer acoes
while True:
    event, values = window.read()
    if event in (None, 'Cancel'):# Termina a execucao do programa
        window.close()
        sys.exit()
        
    elif (event == 'Continue'): #Continua a execucao do programa
        break
        
window.close()

inter1x=int(values["inter1x"])
inter2x=int(values["inter2x"])
inter1y=int(values["inter1y"])
inter2y=int(values["inter2y"])

PrmXname=values["PInames"]

SystemExit: 

2020-06-12 21:24:10,495 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 99 ¦ /home/daniel/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [ ]:

layout = [ [sg.Text('Aperte para finalizar')],
    [sg.Button('Sair')]]
# Cria a janela
window = sg.Window('Leitor Plotter', layout)


    #plota a dataset atualizando a medida que houverem novas medidas

pp = ProgressPlot(x_iterator=False,x_lim=[inter1x, inter2x], y_lim=[inter1y, inter2y])
#pega pontos da base de dados
Y = dataset.get_parameter_data()[PrmYname][PrmYname]
X = dataset.get_parameter_data()[PrmYname][PrmXname]
aLen = len(Y)

#insere os pontos já carregados no grafico
for i in range(0,aLen):
        pp.update(X[i],Y[i])

while 1:
    #pega dados da janela
        event, values = window.read()
        #se o usuario apertar sair encerra
        if event in (None, 'Sair'):# Termina a execucao do programa
                window.close()
                sys.exit()

        Y = dataset.get_parameter_data()[PrmYname][PrmYname]
        X = dataset.get_parameter_data()[PrmYname][PrmXname]

        newLen = len(Y)
        for i in range(aLen,newLen):
            pp.update(X[i],Y[i])

        aLen = newLen
